In [7]:
#Originally sourced : https://machinelearningmastery.com/how-to-develop-deep-learning-models-for-univariate-time-series-forecasting/
import pandas_datareader as dr
import pandas as pd
from datetime import datetime
import pandas_datareader.data as web

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D

from keras.optimizers import SGD
from keras.utils import np_utils
from scipy import misc
import glob

In [8]:
start=datetime(2015,1,1)
end=datetime(2019,4,1)

In [9]:
# tesla = web.DataReader('TSLA', 'google', start,end)
microsoft = dr.data.get_data_yahoo('MSFT', start=start, end=end)

dataset = microsoft
close_data = microsoft['Close']
dates = pd.DataFrame(close_data.index)

dataset.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-01-02,47.419998,46.540001,46.660000,46.759998,27913900.0,42.418739
2015-01-05,46.730000,46.250000,46.369999,46.330002,39673900.0,42.028660
2015-01-06,46.750000,45.540001,46.380001,45.650002,36447900.0,41.411785
2015-01-07,46.459999,45.490002,45.980000,46.230000,29114100.0,41.937946
2015-01-08,47.750000,46.720001,46.750000,47.590000,29645200.0,43.171677


In [ ]:
# evaluate lstm
from math import sqrt
from numpy import array
from numpy import mean
from numpy import std
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from matplotlib import pyplot

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

# transform list into supervised learning format
def series_to_supervised(data, n_in=1, n_out=1):
    df = DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg.values

# root mean squared error or rmse
def measure_rmse(actual, predicted):
    print(actual, predicted)
    return sqrt(mean_squared_error(actual, predicted))

# difference dataset
def difference(data, interval):
    return [data[i] - data[i - interval] for i in range(interval, len(data))]

# fit a model
def model_fit(train, config):
    # unpack config
    n_input, n_nodes, n_epochs, n_batch, n_diff = config
    # prepare data
    if n_diff > 0:
        train = difference(train, n_diff)
    data = series_to_supervised(train, n_in=n_input)
    train_x, train_y = data[:, :-1], data[:, -1]
    train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], 1))
    # define model
    model = Sequential()
    model.add(LSTM(n_nodes, activation='relu', input_shape=(n_input, 1)))
    model.add(Dense(n_nodes, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    # fit
    model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch, verbose=0)
    return model

# forecast with a pre-fit model
def model_predict(model, history, config):
    # unpack config
    n_input, _, _, _, n_diff = config
    # prepare data
    correction = 0.0
    if n_diff > 0:
        correction = history[-n_diff]
        history = difference(history, n_diff)
    x_input = array(history[-n_input:]).reshape((1, n_input, 1))
    # forecast
    yhat = model.predict(x_input, verbose=0)
    return correction + yhat[0]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # fit model
    model = model_fit(train, cfg)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # fit model and make forecast for history
        yhat = model_predict(model, history, cfg)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
    # estimate prediction error
    error = measure_rmse(test, predictions)
    print(' > %.3f' % error)
    return error

# repeat evaluation of a config
def repeat_evaluate(data, config, n_test, n_repeats=30):
    # fit and evaluate the model n times
    scores = [walk_forward_validation(data, n_test, config) for _ in range(n_repeats)]
    return scores

# summarize model performance
def summarize_scores(name, scores):
    # print a summary
    scores_m, score_std = mean(scores), std(scores)
    print('%s: %.3f RMSE (+/- %.3f)' % (name, scores_m, score_std))
    # box and whisker plot
    pyplot.boxplot(scores)
    pyplot.show()

series = dataset['Close']
data = series.values
# data split
n_test = 200
# define config
config = [63, 50, 100, 100, 0]
#n_input, n_nodes, n_epochs, n_batch, n_diff
# grid search
scores = repeat_evaluate(data, config, n_test)
# summarize scores
summarize_scores('lstm', scores)

[100.12999725 100.86000061 100.86000061 101.87000275 101.13999939
 100.41000366  98.38999939  99.08000183  97.54000092  98.62999725
  98.61000061 100.01000214  99.05000305  99.76000214 101.16000366
 101.84999847 102.12000275 101.98000336 104.19000244 105.43000031
 104.91000366 105.94999695 105.12000275 104.40000153 106.26999664
 107.97000122 107.66000366 110.83000183 109.62000275 107.68000031
 105.37000275 106.08000183 106.27999878 107.56999969 108.04000092
 108.12999725 108.87999725 109.48999786 109.66999817 109.
 108.20999908 109.55999756 107.66000366 107.63999939 107.58000183
 106.87000275 105.98000336 107.05999756 107.55999756 108.40000153
 109.59999847 110.26000214 112.01999664 111.94999695 112.33000183
 111.70999908 108.48999786 108.73999786 108.20999908 109.37999725
 111.23999786 111.70999908 112.91000366 113.37000275 112.13999939
 113.20999908 111.69999695 113.56999969 114.26000214 114.66999817
 114.44999695 113.98000336 114.41000366 114.37000275 115.61000061
 115.15000153 115.

[100.12999725 100.86000061 100.86000061 101.87000275 101.13999939
 100.41000366  98.38999939  99.08000183  97.54000092  98.62999725
  98.61000061 100.01000214  99.05000305  99.76000214 101.16000366
 101.84999847 102.12000275 101.98000336 104.19000244 105.43000031
 104.91000366 105.94999695 105.12000275 104.40000153 106.26999664
 107.97000122 107.66000366 110.83000183 109.62000275 107.68000031
 105.37000275 106.08000183 106.27999878 107.56999969 108.04000092
 108.12999725 108.87999725 109.48999786 109.66999817 109.
 108.20999908 109.55999756 107.66000366 107.63999939 107.58000183
 106.87000275 105.98000336 107.05999756 107.55999756 108.40000153
 109.59999847 110.26000214 112.01999664 111.94999695 112.33000183
 111.70999908 108.48999786 108.73999786 108.20999908 109.37999725
 111.23999786 111.70999908 112.91000366 113.37000275 112.13999939
 113.20999908 111.69999695 113.56999969 114.26000214 114.66999817
 114.44999695 113.98000336 114.41000366 114.37000275 115.61000061
 115.15000153 115.

[100.12999725 100.86000061 100.86000061 101.87000275 101.13999939
 100.41000366  98.38999939  99.08000183  97.54000092  98.62999725
  98.61000061 100.01000214  99.05000305  99.76000214 101.16000366
 101.84999847 102.12000275 101.98000336 104.19000244 105.43000031
 104.91000366 105.94999695 105.12000275 104.40000153 106.26999664
 107.97000122 107.66000366 110.83000183 109.62000275 107.68000031
 105.37000275 106.08000183 106.27999878 107.56999969 108.04000092
 108.12999725 108.87999725 109.48999786 109.66999817 109.
 108.20999908 109.55999756 107.66000366 107.63999939 107.58000183
 106.87000275 105.98000336 107.05999756 107.55999756 108.40000153
 109.59999847 110.26000214 112.01999664 111.94999695 112.33000183
 111.70999908 108.48999786 108.73999786 108.20999908 109.37999725
 111.23999786 111.70999908 112.91000366 113.37000275 112.13999939
 113.20999908 111.69999695 113.56999969 114.26000214 114.66999817
 114.44999695 113.98000336 114.41000366 114.37000275 115.61000061
 115.15000153 115.

[100.12999725 100.86000061 100.86000061 101.87000275 101.13999939
 100.41000366  98.38999939  99.08000183  97.54000092  98.62999725
  98.61000061 100.01000214  99.05000305  99.76000214 101.16000366
 101.84999847 102.12000275 101.98000336 104.19000244 105.43000031
 104.91000366 105.94999695 105.12000275 104.40000153 106.26999664
 107.97000122 107.66000366 110.83000183 109.62000275 107.68000031
 105.37000275 106.08000183 106.27999878 107.56999969 108.04000092
 108.12999725 108.87999725 109.48999786 109.66999817 109.
 108.20999908 109.55999756 107.66000366 107.63999939 107.58000183
 106.87000275 105.98000336 107.05999756 107.55999756 108.40000153
 109.59999847 110.26000214 112.01999664 111.94999695 112.33000183
 111.70999908 108.48999786 108.73999786 108.20999908 109.37999725
 111.23999786 111.70999908 112.91000366 113.37000275 112.13999939
 113.20999908 111.69999695 113.56999969 114.26000214 114.66999817
 114.44999695 113.98000336 114.41000366 114.37000275 115.61000061
 115.15000153 115.

[100.12999725 100.86000061 100.86000061 101.87000275 101.13999939
 100.41000366  98.38999939  99.08000183  97.54000092  98.62999725
  98.61000061 100.01000214  99.05000305  99.76000214 101.16000366
 101.84999847 102.12000275 101.98000336 104.19000244 105.43000031
 104.91000366 105.94999695 105.12000275 104.40000153 106.26999664
 107.97000122 107.66000366 110.83000183 109.62000275 107.68000031
 105.37000275 106.08000183 106.27999878 107.56999969 108.04000092
 108.12999725 108.87999725 109.48999786 109.66999817 109.
 108.20999908 109.55999756 107.66000366 107.63999939 107.58000183
 106.87000275 105.98000336 107.05999756 107.55999756 108.40000153
 109.59999847 110.26000214 112.01999664 111.94999695 112.33000183
 111.70999908 108.48999786 108.73999786 108.20999908 109.37999725
 111.23999786 111.70999908 112.91000366 113.37000275 112.13999939
 113.20999908 111.69999695 113.56999969 114.26000214 114.66999817
 114.44999695 113.98000336 114.41000366 114.37000275 115.61000061
 115.15000153 115.

...

[100.12999725 100.86000061 100.86000061 101.87000275 101.13999939
 100.41000366  98.38999939  99.08000183  97.54000092  98.62999725
  98.61000061 100.01000214  99.05000305  99.76000214 101.16000366
 101.84999847 102.12000275 101.98000336 104.19000244 105.43000031
 104.91000366 105.94999695 105.12000275 104.40000153 106.26999664
 107.97000122 107.66000366 110.83000183 109.62000275 107.68000031
 105.37000275 106.08000183 106.27999878 107.56999969 108.04000092
 108.12999725 108.87999725 109.48999786 109.66999817 109.
 108.20999908 109.55999756 107.66000366 107.63999939 107.58000183
 106.87000275 105.98000336 107.05999756 107.55999756 108.40000153
 109.59999847 110.26000214 112.01999664 111.94999695 112.33000183
 111.70999908 108.48999786 108.73999786 108.20999908 109.37999725
 111.23999786 111.70999908 112.91000366 113.37000275 112.13999939
 113.20999908 111.69999695 113.56999969 114.26000214 114.66999817
 114.44999695 113.98000336 114.41000366 114.37000275 115.61000061
 115.15000153 115.

[100.12999725 100.86000061 100.86000061 101.87000275 101.13999939
 100.41000366  98.38999939  99.08000183  97.54000092  98.62999725
  98.61000061 100.01000214  99.05000305  99.76000214 101.16000366
 101.84999847 102.12000275 101.98000336 104.19000244 105.43000031
 104.91000366 105.94999695 105.12000275 104.40000153 106.26999664
 107.97000122 107.66000366 110.83000183 109.62000275 107.68000031
 105.37000275 106.08000183 106.27999878 107.56999969 108.04000092
 108.12999725 108.87999725 109.48999786 109.66999817 109.
 108.20999908 109.55999756 107.66000366 107.63999939 107.58000183
 106.87000275 105.98000336 107.05999756 107.55999756 108.40000153
 109.59999847 110.26000214 112.01999664 111.94999695 112.33000183
 111.70999908 108.48999786 108.73999786 108.20999908 109.37999725
 111.23999786 111.70999908 112.91000366 113.37000275 112.13999939
 113.20999908 111.69999695 113.56999969 114.26000214 114.66999817
 114.44999695 113.98000336 114.41000366 114.37000275 115.61000061
 115.15000153 115.

[100.12999725 100.86000061 100.86000061 101.87000275 101.13999939
 100.41000366  98.38999939  99.08000183  97.54000092  98.62999725
  98.61000061 100.01000214  99.05000305  99.76000214 101.16000366
 101.84999847 102.12000275 101.98000336 104.19000244 105.43000031
 104.91000366 105.94999695 105.12000275 104.40000153 106.26999664
 107.97000122 107.66000366 110.83000183 109.62000275 107.68000031
 105.37000275 106.08000183 106.27999878 107.56999969 108.04000092
 108.12999725 108.87999725 109.48999786 109.66999817 109.
 108.20999908 109.55999756 107.66000366 107.63999939 107.58000183
 106.87000275 105.98000336 107.05999756 107.55999756 108.40000153
 109.59999847 110.26000214 112.01999664 111.94999695 112.33000183
 111.70999908 108.48999786 108.73999786 108.20999908 109.37999725
 111.23999786 111.70999908 112.91000366 113.37000275 112.13999939
 113.20999908 111.69999695 113.56999969 114.26000214 114.66999817
 114.44999695 113.98000336 114.41000366 114.37000275 115.61000061
 115.15000153 115.

[100.12999725 100.86000061 100.86000061 101.87000275 101.13999939
 100.41000366  98.38999939  99.08000183  97.54000092  98.62999725
  98.61000061 100.01000214  99.05000305  99.76000214 101.16000366
 101.84999847 102.12000275 101.98000336 104.19000244 105.43000031
 104.91000366 105.94999695 105.12000275 104.40000153 106.26999664
 107.97000122 107.66000366 110.83000183 109.62000275 107.68000031
 105.37000275 106.08000183 106.27999878 107.56999969 108.04000092
 108.12999725 108.87999725 109.48999786 109.66999817 109.
 108.20999908 109.55999756 107.66000366 107.63999939 107.58000183
 106.87000275 105.98000336 107.05999756 107.55999756 108.40000153
 109.59999847 110.26000214 112.01999664 111.94999695 112.33000183
 111.70999908 108.48999786 108.73999786 108.20999908 109.37999725
 111.23999786 111.70999908 112.91000366 113.37000275 112.13999939
 113.20999908 111.69999695 113.56999969 114.26000214 114.66999817
 114.44999695 113.98000336 114.41000366 114.37000275 115.61000061
 115.15000153 115.

[100.12999725 100.86000061 100.86000061 101.87000275 101.13999939
 100.41000366  98.38999939  99.08000183  97.54000092  98.62999725
  98.61000061 100.01000214  99.05000305  99.76000214 101.16000366
 101.84999847 102.12000275 101.98000336 104.19000244 105.43000031
 104.91000366 105.94999695 105.12000275 104.40000153 106.26999664
 107.97000122 107.66000366 110.83000183 109.62000275 107.68000031
 105.37000275 106.08000183 106.27999878 107.56999969 108.04000092
 108.12999725 108.87999725 109.48999786 109.66999817 109.
 108.20999908 109.55999756 107.66000366 107.63999939 107.58000183
 106.87000275 105.98000336 107.05999756 107.55999756 108.40000153
 109.59999847 110.26000214 112.01999664 111.94999695 112.33000183
 111.70999908 108.48999786 108.73999786 108.20999908 109.37999725
 111.23999786 111.70999908 112.91000366 113.37000275 112.13999939
 113.20999908 111.69999695 113.56999969 114.26000214 114.66999817
 114.44999695 113.98000336 114.41000366 114.37000275 115.61000061
 115.15000153 115.